<a href="https://colab.research.google.com/github/ilovesylus/computer/blob/main/EX05_03_Reply_Multi_type_Messagesipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 6.8 MB/s eta 0:00:00


In [ ]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
  ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
  ngrok.connect(5000)
  run_with_ngrok(app)


from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,StickerMessage,ImageMessage,VideoMessage,LocationMessage
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
  signature = request.headers['X-Line-Signature']
  body = request.get_data(as_text=True)
  try:
    handler.handle(body, signature)
  except InvalidSignatureError:
    abort(400)
  return 'OK'



import google.generativeai as genai

genai.configure(api_key=userdata.get("GOOGLE_API_KEY"))
model = genai.GenerativeModel("gemini-2.0-flash-exp")
def ask_gemini(question):
  response = model.generate_content(question)
  return response.text


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
  with ApiClient(configuration) as api_client:
    line_bot_api = MessagingApi(api_client)

    action = event.message.text
    if action == 'sticker':
      #回復貼圖
      reply = StickerMessage(
        package_id='446',
        sticker_id='1989'
      )
    elif action == 'image':
      #回復圖片
      reply = ImageMessage(
        original_content_url='https://assets.papegames.com/nikkiweb/papegame/deepspacetw/material/20240705/08f0bef2c9df599d8e600f53526d1d04.jpeg',
        preview_image_url='https://assets.papegames.com/nikkiweb/papegame/deepspacetw/material/20240705/08f0bef2c9df599d8e600f53526d1d04.jpeg'
      )
    elif action == 'video':
      reply = VideoMessage(
        original_content_url='https://assets.papegames.com/nikkiweb/papegame/deepspacetw/material/20240708/276b8e902aec8370c4c429f60092a0be.mp4',
        preview_image_url='https://assets.papegames.com/nikkiweb/papegame/deepspacetw/material/20240708/276b8e902aec8370c4c429f60092a0be.mp4'
      )
    elif action == 'location':
      reply = LocationMessage(
        title='Taipei 101',
        address='110台北市信義區信義路五段7號',
        latitude=25.034199559170606,
        longitude=121.56456035396677
      )
    else:
      response = ask_gemini(action)
      reply = TextMessage(text=response)

    line_bot_api.reply_message(
        ReplyMessageRequest(
            reply_token=event.reply_token,
            messages=[
              reply
            ]
          )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://bd5e-34-138-246-57.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:04:25] "GET / HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:04:25] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:04:35] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:04:43] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:04:51] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:04:58] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:09:53] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:09:57] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:10:06] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:10:28] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:10:41] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/May/2025 15:10:59] "POST /callback HTTP/1.1" 200 -
INFO:werkze